<a href="https://colab.research.google.com/github/wonyoung-ji/NLP/blob/master/8%EC%9D%BC%EC%B0%A8/%ED%86%A0%ED%94%BD%EB%AA%A8%EB%8D%B8%EB%A7%81_%EC%8B%A4%EC%8A%B52_LDA(KOR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치 및 드라이브 연동

In [ ]:
# 나눔폰트 설치
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 런타임 다시 시작

In [ ]:
! sudo apt-get install g++ openjdk-7-jdk #Install Java 1.7+ 
#!sudo apt-get install python-dev; pip install konlpy # Python 2.x 
!sudo apt-get install python3-dev; pip3 install konlpy # Python 3.x 
!sudo apt-get install curl 
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# 데이터 전처리

## 데이터 불러오기

In [117]:
import pandas as pd
lda_df = pd.read_csv('/content/gdrive/My Drive/NLP/LDA_example.csv')
lda_df.head()

,content
0,후불정산 방식 악용해 서버 혹사 시키고 잠적 비트코인 등 가상통화 광풍으로 국내 클...
1,개인정보 유출 공지 안내 이메일로 위장한 '가상화폐 채굴 악성코드'가 나타났다. P...
2,행정안전부 국가기록원은 9일부터 국민의 열람 수요가 많은 서울 경기권 지적원도 약 ...
3,"소비자잡지 초이스, 추천제품 선정 LG전자 건조기가 호주서 호평을 받았다.LG전자 ..."
4,스타크래프트 AI대회 韓 첫 1위데이비드 처칠 David Churchill 캐나다 ...


## 품사부착

In [118]:
lda_df['content'] = lda_df['content'].str.replace("[^가-힣ㄱ-ㅎㅏ-ㅣ|\\s]",' ')                       # 한글 이외의 단어 제외
lda_df['content'] = lda_df['content'].apply(lambda x:' '.join([ w for w in x.split() if len(w)>1]))   # 1글자 삭제
tokenized_doc = lda_df['content'].apply(lambda x: mecab.pos(x))                                       # 품사태깅
tokenized_doc

0     [(후불, NNG), (정산, NNG), (방식, NNG), (악용, NNG), (...
1     [(개인, NNG), (정보, NNG), (유출, NNG), (공지, NNG), (...
2     [(행정, NNG), (안, MAG), (전부, MAG), (국가, NNG), (기...
3     [(소비자, NNG), (잡지, NNG), (초이스, NNP), (추천, NNG),...
4     [(스타크래프트, NNP), (대회, NNG), (위, NNG), (데이비드, NN...
5     [(성균관, NNP), (대, XPN), (연구, NNG), (팀, NNG), (표...
6     [(손건재, NNP), (포스코, NNP), (신임, NNG), (대표, NNG),...
7     [(국내, NNG), (통신사, NNG), (유일, NNG), (하, XSA), (...
8     [(반도체, NNG), (경쟁력, NNG), (강화, NNG), (와, JC), (...
9     [(하이닉스, NNP), (지난, VV+ETM), (일, NNG), (부터, JX)...
10    [(보도, NNG), (합니다, XSV+EF), (손등, NNG), (얇, VA),...
11    [(팹, NNP), (리스, NNP), (기업, NNG), (인, VCP+ETM),...
12    [(자사, NNG), (의, JKG), (인공지능, NNP), (서비스, NNG),...
13    [(출고가, NNG), (자기, NP), (부상, NNG), (디자인, NNG), ...
14    [(모바일, NNP), (플랫폼, NNG), (서비스, NNG), (회사, NNG)...
15    [(사진, NNG), (서울산, NNP), (업진, NNG), (흥원, NNP), ...
16    [(한국, NNP), (중견, NNG), (기업, NNG), (연합회, NNG), ...
17    [(지난, VV+ETM), (구간, NNG), (데이터, NNG), (무제,

## 불용어처리

In [119]:
from konlpy.tag import Mecab
mecab = Mecab()
stop_pos=['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX',   #조사
           'SF','SE','SSO','SSC','SC','SH','SL','SN','SY',     #기타부호
           'VCP','VCN',
           'EP','EF','ETN','ETM',    #어미
           'XSN','XSV','XSA','XPN',]
          
stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']

words=[]
for tags in tokenized_doc:
  word = []
  for tag in tags:
    if tag[1] not in stop_pos:
      if len(tag[0]) > 1:
        word.append(tag[0])
  words.append(' '.join(word))

tokenized_doc=words

In [120]:
tokenized_doc[:3]

['후불 정산 방식 악용 서버 혹사 비트코인 가상 통화 광풍 국내 클라우드 업계 피해 비트코인 채굴 과정 성능 그래픽 카드 비싼 하드웨어 필요 한데 부담 줄이 위해 클라우드 서비스 통해 가상 통화 채굴 변칙 사례 증가 때문 관련 업계 따르 엔터 토스트 클라우드 다음 사용 약관 가상 통화 채굴 금지 조항 신설 키로 채굴 단기간 엄청난 전산 자원 만큼 정상 서비스 이용 어렵 클라우드 후불 정산 방식 악용 채굴 서버 혹사 정산 절차 없이 잠적 사례 전해진다 네이버 약관 개정 판촉 레딧 무료 서버 써서 가상 통화 채굴 행위 전면 금지 업계 에선 국내 클라우드 업체 가상 통화 채굴 대책 마련 사례 계속 전망 외국 클라우드 기반 가상 화폐 채굴 문제 올해 해커 아마존 서비스 기업 계정 빼돌려 가상 통화 채굴 악용 사실 드러났 가상 통화 전산 암호 기술 토대 사이버 화폐 중앙 발급 기관 국외 송금 비용 매우 특징 실제 채굴 개인 성능 거래 인증 가상 통화 전산 운영 참여 행위 채굴 이런 기여 대가 일정량 화폐 는다 세상 나올 비트코인 한정 채굴 할수록 효율',
 '개인 정보 유출 공지 안내 이메일 위장 가상 화폐 채굴 코드 나타났 만든 오프라인 가상 화폐 지갑 탈취 공격 발견 가상 화폐 투자자 물론 일반인 가상 화폐 노린 해커 희생자 업계 개인 정보 빼낸 해커 고객 이메일 보내 악성 코드 감염 신종 수법 경고 가상 화폐 투자 열기 가운데 채굴 컴퓨팅 파워 확보 시도 보인다',
 '행정 전부 국가 기록원 국민 열람 수요 서울 경기 지적 원도 공공누리 유형 표시 내려받기 기능 제공 한다고 밝혔 공공누리 국가 지방 자치 단체 공공 기관 저작물 민간 자유 활용 도록 이용 허락 범위 규정 한다 예외 공공 기관 필요 따라 상업 이용 금지 변경 금지 조건 부여 된다 국가 기록원 일제 강점기 생산 지역 전체 지적 원도 일부 사진 필름 기록물 대해 공공누리 서비스 추진 계획 먼저 국민 열람 수요 서울 경기 지적 원도 공개 시작 강원 충청 경상 전라 지적 원도 차례 공공누리 유형 표시 내려

# LDA

In [126]:
def my_tokenizer(text):
  return mecab.morphs(text)

In [130]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tfidf_vect = TfidfVectorizer(tokenizer=my_tokenizer)
tfidf = tfidf_vect.fit_transform(tokenized_doc)
lda = LatentDirichletAllocation(n_components=3, max_iter=20, learning_method='online')


lda_output = lda.fit_transform(tfidf)

In [ ]:
!pip install pyLDAvis

In [132]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne')
pyLDAvis.display(vis)